일별박스오피스데이터 폴더 속의 일별 박스오피스 데이터를 전부 취합한다  
이후 각 영화별로 날짜를 정리한다

In [1]:
import pandas as pd
import os
import glob
from tqdm import tqdm
import re

In [2]:
movie_folder = "일별박스오피스데이터"
search_pattern = os.path.join(movie_folder, '*.csv')
file_list = glob.glob(search_pattern)

In [3]:
full_data = pd.DataFrame()  # 모든 데이터를 담을 빈 데이터프레임 생성
for file_path in tqdm(file_list):
    file_name = os.path.basename(file_path)[:-4] #일별 날짜
    temp=pd.read_csv(file_path, encoding='utf-8-sig') #각 일자별 박스오피스 데이터
    temp=temp[["영화명","개봉일","매출액", "관객수", "스크린수","상영횟수","대표국적","배급사","등급","장르","감독","배우"]]
    temp['일자']=file_name
    full_data = pd.concat([full_data, temp], ignore_index=True)
    
    


100%|██████████| 7970/7970 [06:30<00:00, 20.42it/s]


In [4]:
full_data.tail()

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,일자
681081,미션 임파서블: 파이널 레코닝,2025-05-17,8000,1,1,1,미국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,액션,크리스토퍼 맥쿼리,"톰 크루즈,헤일리 앳웰,빙 라메스,사이먼 페그,바네사 커비,에사이 모랄레스,폼 클레...",2025-09-06
681082,이빨요정 비올레타: 요정나라로 돌아갈래!,2023-11-18,0,0,1,1,독일,TCO(주)더콘텐츠온,전체관람가,"애니메이션,판타지,어드벤처",NaN,NaN,2025-09-06
681083,해피엔드,2025-04-30,0,0,1,1,일본,"(주)영화사 진진,(주)스튜디오 산타클로스엔터테인먼트",15세이상관람가,드라마,소라 네오,"쿠리하라 하야토,히다카 유키토,하야시 유타,시나 펭,아라지,이노리 키라라,나카지마 아유무",2025-09-06
681084,파과,2025-04-30,0,0,1,1,한국,(주)넥스트엔터테인먼트월드(NEW),15세이상관람가,"액션,드라마,미스터리",민규동,"이혜영,김성철,연우진,김무열,신시아,옥자연,김강우,최무성,양주미,양주미",2025-09-06
681085,딸에 대하여,2024-09-04,0,0,1,1,한국,"찬란,스튜디오 에이드",12세이상관람가,드라마,이미랑,"오민애,허진,임세미,하윤경,신정섭",2025-09-06


In [16]:
full_data.to_csv("원본_박스오피스_데이터.csv", index=False, encoding='utf-8-sig')

## 영화 목록 추출

In [5]:
movie_list=pd.DataFrame(list(full_data["영화명"].unique()), columns=['영화명'])

In [6]:
movie_list.to_csv("영화목록.csv", index=False, encoding='utf-8-sig')

In [7]:
movie_list['영화명'][movie_list['영화명']=='동생 몰래 잔뜩 맛본 제부의 맛 무삭제판'].index

Index([39995], dtype='int64')

영화 목록에 번호 추가 (인덱스용)

In [3]:
import pandas as pd
temp_list=pd.read_csv("영화목록.csv", encoding="utf-8-sig")
temp_list["인덱스"]=temp_list.index+1
temp_list.to_csv("영화목록.csv", index=False, encoding='utf-8-sig')

## 영화별 추출

In [15]:
# 파일명으로 사용할 수 없는 특수문자를 제거하는 함수
def clean_filename(name):
    # 정규표현식을 사용하여 파일명에 부적합한 문자를 빈 문자열('')로 치환
    # 제거할 문자: \ / : * ? " < > | \t \n \r
    cleaned_name = re.sub(r'[\\/*?:"<>|!\t\n\r]', '', name)
    return cleaned_name

# 재개봉 판단 로직 함수
def get_release_label(group_id):
    if group_id == 1:
        return '최초 개봉'
    else:
        return f'{group_id - 1}차 재개봉'
    
# tqdm 라이브러리를 사용하여 진행 상황을 확인하며 파일 저장
for movie_title in tqdm(movie_list['영화명']):
    # 1. 원본 영화 제목으로 데이터 필터링
    temp_movie_data = full_data[full_data['영화명'] == movie_title].copy()

    #재개봉 판단 로직
    threshold = pd.Timedelta(days=240) #임계치 8개월
    temp_movie_data["일자"] = pd.to_datetime(temp_movie_data["일자"], errors='coerce')
    date_diffs = temp_movie_data['일자'].diff()
    release_group_id = (date_diffs > threshold).cumsum() + 1
    temp_movie_data['개봉 구분'] = release_group_id.apply(get_release_label)

    
    # 2. 파일명에 부적합한 문자를 제거
    safe_filename = clean_filename(movie_title)
    
    # 3. 안전한 파일명으로 CSV 저장
    temp_movie_data.to_csv(f"영화별박스오피스데이터/{safe_filename}.csv", index=False, encoding='utf-8-sig')

100%|██████████| 2700/2700 [01:50<00:00, 24.45it/s]


## 전처리한 모든 박스오피스 데이터 통합

In [17]:
result=pd.DataFrame()

In [18]:
movie_folder = "영화별박스오피스데이터"
search_pattern = os.path.join(movie_folder, '*.csv')
file_list = glob.glob(search_pattern)

In [23]:
for file_path in tqdm(file_list):
    temp=pd.read_csv(file_path, encoding='utf-8-sig') #각 일자별 박스오피스 데이터
    result = pd.concat([result, temp], ignore_index=True)

100%|██████████| 42696/42696 [52:31<00:00, 13.55it/s]


In [24]:
result.to_csv("최종_박스오피스_데이터.csv", index=False, encoding='utf-8-sig')